In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from typing import Union
from langchain.messages import HumanMessage, AIMessage, ToolCall, ToolMessage
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: List[BaseMessage]
    context: str


In [ ]:
from typing import Dict
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from pydantic import Field


@tool
def update_user_context(content: str) -> str:
    """Use this to update the user's context memory, the existing memory will be replaced by your content"""

    try:
        with open("memory/user_profile.md", "w") as f:
            f.write(content)
    except Exception as e:
        return f"Error: {e}"

    return f"Succesfully updated the memory"

@tool
def add_syllabus(file_name: str, content: Dict) -> str:
    """This function saves a new syllabus in the memory"""

    try:
        if not file_name.endswith(".json"):
            file_name = file_name[:file_name.rfind(".")] + ".json"
        with open(f"memory/syllabuses/{file_name}", "w") as f:
            f.write(content)

    except Exception as e:
        return f"Error: {e}"
    return "Succesfully saved a new syllabus"


@tool
def get_all_syllabus() -> List[str]:
    """It will return all of the saved syllabuses' names"""

    try:
        return os.listdir("memory/syllabuses/")
    except Exception as e:
        return f"Error: {e}"

@tool
def get_syllabus(file_name: str) -> str:
    """It will return the content of the provided syllabus"""

    try:
        with open(f"memory/syllabuses/{file_name}", "r") as f:
            return f.read()

    except Exception as e:
        return f"Error {e}"

tools = [update_user_context, add_syllabus, get_all_syllabus, get_syllabus]

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

llm_tools = llm.bind_tools(tools)


In [5]:
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate

config = {"configurable": {"thread_id": "customer_123"}}

with open("memory/user_profile.md", "r") as f:
    context_memory = f.read()

prompt_template = ChatPromptTemplate(
    [
        ("system", "You are personal assistent, you help to manage the user's academic issues"),
        ("human", "The user's context:\n{context_memory}\n\nthe user's query:\n{text}")
    ]
)

prompt_value = prompt_template.invoke(
    {
        "context_memory": context_memory,
        "text": "save to the memory that i like Linear Algebra"
    }
)

agent = create_agent(llm, tools)


In [ ]:

"""Бесконечный цикл общения с агентом. Выход: exit, quit или q"""
from langchain_core.messages import SystemMessage

config = {"configurable": {"thread_id": "customer_123"}}
history = []  # сохранённая история: HumanMessage, AIMessage, ToolMessage, ...

print("Uni-Agent готов к общению. Введите 'exit', 'quit' или 'q' для выхода.\n")

while True:
    user_input = input("Вы: ").strip()
    if not user_input:
        continue
    if user_input.lower() in ("exit", "quit", "q"):
        print("До свидания!")
        break
    
    # Загружаем контекст (может обновиться после update_user_context)
    with open("memory/user_profile.md", "r") as f:
        context_memory = f.read()
    
    system_content = (
        "You are personal assistant, you help to manage the user's academic issues.\n\n"
        f"The user's context:\n{context_memory}"
    )
    
    messages = [SystemMessage(content=system_content)] + history + [HumanMessage(content=user_input)]
    
    result = agent.invoke({"messages": messages}, config=config)
    
    # Новые сообщения от агента (после нашего HumanMessage)
    new_messages = result["messages"][len(messages):]
    history = history + [HumanMessage(content=user_input)] + new_messages
    
    # Печатаем последний текстовый ответ
    for m in new_messages:
        m.pretty_print()
        


Uni-Agent готов к общению. Введите 'exit', 'quit' или 'q' для выхода.


Ассистент: Количество пропусков по предмету "Казахстан Тарихы" может зависеть от политики конкретного учебного заведения и курса. Обычно, правила о пропусках могут включать:

1. Определенное количество пропусков без справки (например, 3-5 занятий).
2. После превышения этого количества, может потребоваться медицинская справка или другое подтверждение.

Рекомендую обратиться к преподавателю или в учебный офис вашего факультета для получения точной информации о правилах пропусков на вашем курсе.


Ассистент: В силлабусе по курсу "История Казахстана" указано, что максимальное количество пропусков составляет 3 занятия. Если вы пропустите больше, это может повлиять на вашу успеваемость. 

Если у вас есть дополнительные вопросы, вы можете обратиться к преподавателю по электронной почте: ivanov@kbtu.kz.


Ассистент: Я обновил информацию о вас. Теперь в контексте указано, что максимальное количество пропусков по курсу "Исто

# Outlook Monitoring

In [ ]:
from IMAP_monitor.email_monitor import OutlookEmailMonitor

KBTU_EMAIL = "n_ashekei@kbtu.kz"
KBTU_PASSWORD = "555999Www"

email_monitor = OutlookEmailMonitor(KBTU_EMAIL, KBTU_PASSWORD)

In [15]:
email_monitor.connect()

Connecting to outlook.office365.com...
✗ Connection failed: b'LOGIN failed.'


False

In [ ]:
application_client_id ="04ef8ab3-74b4-45e3-a7d0-7a7023f78396"


# Application (client) ID
# :
# 04ef8ab3-74b4-45e3-a7d0-7a7023f78396
# CMD+C to copy
# Copy to clipboard the value for Application (client) ID 04ef8ab3-74b4-45e3-a7d0-7a7023f78396
# Object ID
# :
# fef5ad23-ded7-4f40-b573-6c6504715434
# CMD+C to copy
# Copy to clipboard the value for Object ID fef5ad23-ded7-4f40-b573-6c6504715434
# Directory (tenant) ID
# :
# 57081b5e-e66a-4993-8eaf-15b0b309293f